In [2]:
# Libraries
import os
import pandas as pd
import csv

#SQL
import sqlite3
from sqlite3 import Error

In [3]:
# Load CSV:
poke_df = pd.read_csv('../data/fullgen_tophalf_pokemon.csv')
poke_df.index.name='id'
poke_df.head()

,hp,attack,defense,special-attack,special-defense,speed,type_1,type_2,Pokemon,totalstat,generation,avg_power
id,,,,,,,,,,,,
0,80,82,83,100,100,80,Grass,Poison,Venusaur,525,1,91
1,78,84,78,109,85,100,Fire,Flying,Charizard,534,1,96
2,79,83,100,85,105,78,Water,NaN,Blastoise,530,1,84
3,83,80,75,70,70,101,Normal,Flying,Pidgeot,479,1,75
4,60,90,55,90,80,110,Electric,NaN,Raichu,485,1,90


In [8]:
connection = create_connection("../data/project.sqlite")
cursor = connection.cursor()

Connection to SQLite DB successful


In [10]:
# Setup standard functions:
def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")
        
def execute_read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as e:
        print(f"The error '{e}' occurred")

In [11]:
drop_pokemon_table = """
DROP TABLE IF EXISTS pokemon_gen1
;"""

create_pokemon_table = """
CREATE TABLE IF NOT EXISTS pokemon_gen1 (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  hp INTEGER,
  attack INTEGER,
  defence INTEGER,
  special_attack INTEGER,
  special_defence INTEGER,
  speed INTEGER,
  type_1 TEXT,
  type_2 TEXT,
  name TEXT,
  totalstat INTEGER
);
"""
execute_query(connection, drop_pokemon_table)
execute_query(connection, create_pokemon_table)

Query executed successfully
Query executed successfully


In [12]:
poke_df.to_sql('top_pokemon', con=connection, if_exists='replace', index=True, index_label=None)

507

In [13]:
select_all = "SELECT * FROM top_pokemon LIMIT 5"
execute_read_query(connection, select_all)

[(0, 80, 82, 83, 100, 100, 80, 'Grass', 'Poison', 'Venusaur', 525, 1, 91),
 (1, 78, 84, 78, 109, 85, 100, 'Fire', 'Flying', 'Charizard', 534, 1, 96),
 (2, 79, 83, 100, 85, 105, 78, 'Water', None, 'Blastoise', 530, 1, 84),
 (3, 83, 80, 75, 70, 70, 101, 'Normal', 'Flying', 'Pidgeot', 479, 1, 75),
 (4, 60, 90, 55, 90, 80, 110, 'Electric', None, 'Raichu', 485, 1, 90)]